In [3]:
pip install -U sentence-transformers

  Using cached sentence_transformers-2.5.1-py3-none-any.whl.metadata (11 kB)
Using cached sentence_transformers-2.5.1-py3-none-any.whl (156 kB)
Note: you may need to restart the kernel to use updated packages.


In [4]:
import pandas as pd
import numpy as np 
from sentence_transformers import SentenceTransformer
from sentence_transformers import  util
import math
from scipy.stats import spearmanr
from sklearn.metrics import r2_score
from sentence_transformers import InputExample, losses
from torch.utils.data import DataLoader

In [16]:
def Modeling(val_data,train_data1,training_data2,mode = "easy",model_name = 'all-distilroberta-v1'):
    """
    This function is meant to preform modeling 
    val_data: a data frame that meant to be the the test data for our model
    train_data1 : a data frame that meant to be first training data set for our model
    train_data2 : a data frame that meant to be first training data set for our model
    mode : the grade_mode for our Gradanizer function  {grade_mode: how fair you want to model to be [fair: the exact transformation with out 
    any lose ranges, easy: more skewed into higher grades more common, lose_ends: more skewed into higher grades and lower grades]} 
    model_name : is the model name form sentance transfomers that you want the modeling to be about default is 'all-distilroberta-v1' because
    its the best currently for our model
    """
    def Gradanizer(num, grade_mode="Fair"):
        """
        This function take the garde form 0 to 1 and output that garde in form 0 to 5
        parameters
        num: the garde in form 0 to 1
        grade_mode: how fair you want to model to be [fair: the exact transformation with out any lose ranges, easy: more skewed into higher grades more 
        common, lose_ends: more skewed into higher grades and lower grades]
        """
        def get_region_value_f(number):    
            intervals = [
            (-10, 0.4545), (0.4545, 0.9091), (0.9091, 1.3636),
            (1.3636, 1.8182), (1.8182, 2.2727), (2.2727, 2.7273),
            (2.7273, 3.1818), (3.1818, 3.6364), (3.6364, 4.0909),
            (4.0909, 4.5455), (4.5455, 6)]
            values=[0,0.5,1,1.5,2,2.5,3,3.5,4,4.5,5]
            for i, (start, end) in enumerate(intervals):
                if start <= number < end:
                    return values[i]
        
        def get_region_value_e(number):    
            intervals = [
            (-10, 0.4545), (0.4545, 0.9091), (0.9091, 1.3636),
            (1.3636, 1.8182), (1.8182, 2.2727), (2.2727, 2.7273),
            (2.7273, 3.1818), (3.1818, 3.6364)]
            values=[0,0.5,1,1.5,2,2.5,3,3.5]
            for i, (start, end) in enumerate(intervals):
                if start <= number < end:
                    return values[i]
        def get_region_value_l(number):    
            intervals = [
            (1.0000, 1.3571), (1.3571, 1.7143), (1.7143, 2.0714),
            (2.0714, 2.4286), (2.4286, 2.7857), (2.7857, 3.1429),
            (3.1429, 3.5000)]
            values=[0.5,1,1.5,2,2.5,3,3.5]
            for i, (start, end) in enumerate(intervals):
                if start <= number < end:
                    return values[i]

        if (grade_mode == "fair"):
            num_f=num*5
            return get_region_value_f(num_f)
        elif (grade_mode == "easy"):
            if (num >=0.85):
                return 5
            elif ( 0.85>num>=0.8):
                return 4.5
            elif ( 0.8>num>=0.7):
                return 4
            else:
                num_e = num*5
                return get_region_value_e(num_e)
        elif (grade_mode == "lose_ends"):
            if (num >=0.85):
                return 5
            elif ( 0.85>num>=0.8):
                return 4.5
            elif ( 0.8>num>=0.7):
                return 4
            elif (0.2>=num):
                return 0
            else:
                num_l = num*5
                return get_region_value_l(num_l)
        else:
            print("not a valid mode ")
            
            
    def Evaluation(embeddings_1=None, embeddings_2=None,test_data=None,mode="easy"):
        """
        this function takes two lists of embeddings in the form of torch tensor or list or df column one for the student answer and other for model answer
        or any two sentences if this function used in any other code and the test data frame and the mode of Gradazier and output MAE % and r_squared and 
        correlation_coefficient
        parameters
        embeddings_1:list or tensor or df column that hold the sentence embeddings (student answer embeddings)
        embeddings_2:list or tensor or df column that hold the sentence embeddings (model answer embeddings)
        data: the test data frame
        mode: the mode of which gradazier is used ["fair", "easy", "lose_ends"]
        """
        from sentence_transformers import util
        if (len(embeddings_1) != len(embeddings_2)):
            print("embeddings_1 and embeddings_2 are not the same length")
            return None
        elif (mode not in ["easy","lose_ends","fair"]):
            print("not valid mode")
            return None
        else:
            predicted = []
            for i in range(len(embeddings_1)):
                predicted.append(Gradanizer(util.cos_sim(embeddings_1[i], embeddings_2[i]),mode))
            transformed_grade=[]
            for i in range(len(embeddings_1)):
                #transformed_grade.append(Gradanizer(test_data["grade"][i],mode))
                transformed_grade.append(test_data["grade"][i]*5)
            arr_predicted =np.array(predicted)
            arr_grade=np.array(transformed_grade)
            MAE = (1-((np.sum(np.abs(arr_predicted-arr_grade))/len(embeddings_1))/5))*100
            correlation_coefficient, p_value = spearmanr(arr_grade, arr_predicted)
            r_squared = r2_score(arr_grade, arr_predicted)
            dif =arr_predicted-arr_grade
            hits_precentage= (np.count_nonzero(dif == 0)/len(embeddings_1))
            #return MAE,correlation_coefficient,r_squared
            #if you want the hit precentage tag the above and untag the blew
            return MAE,correlation_coefficient,hits_precentage
        
    def data_loader_local(data=None):
        input_data = []
        for index, row in data.iterrows():
            student_answer = row['student_answer']
            model_answer = row['model_answer']
            grade = row['grade']
        
            # Create InputExample instances
            input_data.append(InputExample(texts=[student_answer, model_answer], label=grade))
        return(input_data)
        
    final_model = SentenceTransformer(model_name)
    
    train_dataset_og = data_loader_local(train_data1)
    train_dataloader_og = DataLoader(train_dataset_og, shuffle=True, batch_size=16)
    train_loss_og = losses.CosineSimilarityLoss(model=final_model)
    final_model.fit(train_objectives=[(train_dataloader_og, train_loss_og)], epochs=5, warmup_steps=330)
    train_dataset_asag = data_loader_local(training_data2)
    train_dataloader_asag = DataLoader(train_dataset_asag, shuffle=True, batch_size=16)
    train_loss_asag = losses.CosineSimilarityLoss(model=final_model)
    final_model.fit(train_objectives=[(train_dataloader_asag, train_loss_asag)], epochs=1, warmup_steps=27)
    students_embed_asag=final_model.encode(val_data["student_answer"].to_list(),convert_to_tensor=True)
    model_answer_embed_asag=final_model.encode(val_data["model_answer"].to_list(),convert_to_tensor=True)
    precentage_2,correlation_coefficient_2,hit_precentage_2 = Evaluation(students_embed_asag,model_answer_embed_asag,val_data,mode)
    return precentage_2,correlation_coefficient_2,hit_precentage_2,final_model

            

            
        


In [6]:
val_data = pd.read_excel(r"/kaggle/input/cleaned-asag-data/Val_data_set.xlsx")

In [7]:
val_data.head()

,question,model_answer,student_answer,grade,source,ID
0,explain classification and regression; what is...,classification is a task of mapping data to di...,classification is type of problem where algori...,0.5,asag_dataset.csv,2
1,describe: the structure on an can.,convolutional neural network consists of many ...,a can consists of one or more convolution laye...,0.5,asag_dataset.csv,2
2,what role does the method of steepest decent ...,steepest descent is used to update the weights...,steepest decent method helps in making the adj...,0.5,asag_dataset.csv,2
3,explain classification and regression; what is...,classification is a task of mapping data to di...,"classification: in classification, the output ...",0.5,asag_dataset.csv,2
4,"explain back propagation, use the correct tech...",backpropagation lowers the error of a mlp leve...,* back propagation is a steepest decent method...,1.0,asag_dataset.csv,2


In [8]:
training_data1=pd.read_excel(r"/kaggle/input/cleaned-asag-data/Traing_Data_analysis_and_mh.xlsx")

In [9]:
training_data1.head()

,question,model_answer,student_answer,grade,source,ID
0,how are linked lists passed as arguments to a ...,by reference.,linked lists permit insertion and removal of n...,0.3,mohler_dataset_edited.csv,4
1,what is the role of a header-file?,"to store a class interface, including data mem...",it contains reusable source code for use by ot...,0.6,mohler_dataset_edited.csv,4
2,"briefly, how does selection sort work?",it selects the minimum from an array and place...,you select the smallest element and place it o...,1.0,mohler_dataset_edited.csv,4
3,what are the access-specifiers that can be use...,private and public.,"local, global, local-variable, function specific",0.4,mohler_dataset_edited.csv,4
4,what is the number of operations for insertion...,n (the length of the array) operations achieve...,"n operations, where n is the number of items.",0.8,mohler_dataset_edited.csv,4


In [10]:
training_data2=pd.read_excel(r"/kaggle/input/cleaned-asag-data/asag_data_set.xlsx")

In [11]:
training_data2.head()

,question,model_answer,student_answer,grade,source,ID
0,"give a definition for the term ""artificial ne...",a neural network is a massively parallel distr...,a ann is a massively distributed processor. it...,1.0,asag_dataset.csv,2
1,"when learning using steepest descent, explain ...",learning rate controls the speed of the conver...,learning rate controls the speed of the descen...,1.0,asag_dataset.csv,2
2,write down the som learning in pseudo code.,arrange the weights in the required topology a...,given a map layer set random small values for ...,1.0,asag_dataset.csv,2
3,describe: the structure on an can.,convolutional neural network consists of many ...,"in a convolution al neural network, the layer ...",1.0,asag_dataset.csv,2
4,"define: echo state network (est), how are they...",echo state network is a type of recurrent neur...,est are the run recurrent neural network which...,0.0,asag_dataset.csv,2


In [17]:
# example
precentage_2,correlation_coefficient_2,hit_precentage_2,model = Modeling(val_data,training_data1,training_data2)

Epoch:   0%|          | 0/5 [00:00<?, ?it/s]

Iteration:   0%|          | 0/221 [00:00<?, ?it/s]

Iteration:   0%|          | 0/221 [00:00<?, ?it/s]

Iteration:   0%|          | 0/221 [00:00<?, ?it/s]

Iteration:   0%|          | 0/221 [00:00<?, ?it/s]

Iteration:   0%|          | 0/221 [00:00<?, ?it/s]

Epoch:   0%|          | 0/1 [00:00<?, ?it/s]

Iteration:   0%|          | 0/17 [00:00<?, ?it/s]

Batches:   0%|          | 0/30 [00:00<?, ?it/s]

Batches:   0%|          | 0/30 [00:00<?, ?it/s]

In [18]:
print(precentage_2,correlation_coefficient_2,hit_precentage_2)

92.97268907563026 0.8222902224329431 0.5577731092436975


In [19]:
model

SentenceTransformer(
  (0): Transformer({'max_seq_length': 512, 'do_lower_case': False}) with Transformer model: RobertaModel 
  (1): Pooling({'word_embedding_dimension': 768, 'pooling_mode_cls_token': False, 'pooling_mode_mean_tokens': True, 'pooling_mode_max_tokens': False, 'pooling_mode_mean_sqrt_len_tokens': False, 'pooling_mode_weightedmean_tokens': False, 'pooling_mode_lasttoken': False, 'include_prompt': True})
  (2): Normalize()
)

In [20]:
    def Gradanizer(num, grade_mode="Fair"):
        """
        This function take the garde form 0 to 1 and output that garde in form 0 to 5
        parameters
        num: the garde in form 0 to 1
        grade_mode: how fair you want to model to be [fair: the exact transformation with out any lose ranges, easy: more skewed into higher grades more 
        common, lose_ends: more skewed into higher grades and lower grades]
        """
        def get_region_value_f(number):    
            intervals = [
            (-10, 0.4545), (0.4545, 0.9091), (0.9091, 1.3636),
            (1.3636, 1.8182), (1.8182, 2.2727), (2.2727, 2.7273),
            (2.7273, 3.1818), (3.1818, 3.6364), (3.6364, 4.0909),
            (4.0909, 4.5455), (4.5455, 6)]
            values=[0,0.5,1,1.5,2,2.5,3,3.5,4,4.5,5]
            for i, (start, end) in enumerate(intervals):
                if start <= number < end:
                    return values[i]
        
        def get_region_value_e(number):    
            intervals = [
            (-10, 0.4545), (0.4545, 0.9091), (0.9091, 1.3636),
            (1.3636, 1.8182), (1.8182, 2.2727), (2.2727, 2.7273),
            (2.7273, 3.1818), (3.1818, 3.6364)]
            values=[0,0.5,1,1.5,2,2.5,3,3.5]
            for i, (start, end) in enumerate(intervals):
                if start <= number < end:
                    return values[i]
        def get_region_value_l(number):    
            intervals = [
            (1.0000, 1.3571), (1.3571, 1.7143), (1.7143, 2.0714),
            (2.0714, 2.4286), (2.4286, 2.7857), (2.7857, 3.1429),
            (3.1429, 3.5000)]
            values=[0.5,1,1.5,2,2.5,3,3.5]
            for i, (start, end) in enumerate(intervals):
                if start <= number < end:
                    return values[i]

        if (grade_mode == "fair"):
            num_f=num*5
            return get_region_value_f(num_f)
        elif (grade_mode == "easy"):
            if (num >=0.85):
                return 5
            elif ( 0.85>num>=0.8):
                return 4.5
            elif ( 0.8>num>=0.7):
                return 4
            else:
                num_e = num*5
                return get_region_value_e(num_e)
        elif (grade_mode == "lose_ends"):
            if (num >=0.85):
                return 5
            elif ( 0.85>num>=0.8):
                return 4.5
            elif ( 0.8>num>=0.7):
                return 4
            elif (0.2>=num):
                return 0
            else:
                num_l = num*5
                return get_region_value_l(num_l)
        else:
            print("not a valid mode ")

In [21]:
model.encode("Learning a new language boosts cognitive abilities, job prospects, and cultural understanding while enhancing confidence and self-esteem.",convert_to_tensor=True)

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

tensor([ 1.6472e-02,  1.2774e-02,  7.0743e-03, -8.8951e-03, -2.6098e-02,
         4.3210e-02, -1.9003e-02, -6.5710e-02, -2.7721e-03, -6.2631e-02,
        -8.1302e-02,  5.0946e-03,  6.7502e-03, -4.3602e-03,  2.0377e-02,
         6.2946e-02,  2.9855e-02,  2.9199e-02,  2.4137e-02,  1.9160e-03,
        -2.5437e-02, -2.0497e-02, -3.2267e-02,  1.3394e-02, -3.3890e-03,
         3.4942e-02,  2.8221e-02, -4.9432e-02, -4.8840e-02,  1.5346e-02,
        -2.6705e-02,  2.0452e-02, -2.3642e-02, -2.2247e-02,  5.2911e-02,
         4.0696e-02,  1.9797e-02,  3.5088e-03,  2.1237e-02,  6.7395e-02,
        -3.9825e-02,  3.5613e-02, -1.6038e-02, -2.5443e-02,  1.1769e-02,
        -2.4139e-02, -5.0673e-02, -2.0879e-02,  1.8502e-02, -5.2584e-02,
         5.2773e-02, -3.4806e-02,  5.2381e-02, -1.8902e-04, -3.3531e-02,
         1.1922e-02, -1.6378e-02,  3.8581e-02,  3.4453e-02, -1.1915e-02,
         1.3410e-02, -1.0745e-02, -1.1264e-01, -3.1111e-02,  3.1931e-02,
         5.8307e-03,  1.6441e-02, -2.3986e-02,  1.1

In [22]:
def pridector(model, student_answer, model_answer, mode):
    """
    Pridector function take granding model and model and student answer and output a final grade
    
    Parameters 
    model : The grading model note most be Sentance transformers model 
    student_answer : the text of the student answer
    model_answer : the text of the model answer
    mode : the mode of gradanizer
    
    """
    

    def Gradanizer(num, grade_mode="Fair"):
        """
        This function take the garde form 0 to 1 and output that garde in form 0 to 5
        parameters
        num: the garde in form 0 to 1
        grade_mode: how fair you want to model to be [fair: the exact transformation with out any lose ranges, easy: more skewed into higher grades more 
        common, lose_ends: more skewed into higher grades and lower grades]
        """
        def get_region_value_f(number):    
            intervals = [
            (-10, 0.4545), (0.4545, 0.9091), (0.9091, 1.3636),
            (1.3636, 1.8182), (1.8182, 2.2727), (2.2727, 2.7273),
            (2.7273, 3.1818), (3.1818, 3.6364), (3.6364, 4.0909),
            (4.0909, 4.5455), (4.5455, 6)]
            values=[0,0.5,1,1.5,2,2.5,3,3.5,4,4.5,5]
            for i, (start, end) in enumerate(intervals):
                if start <= number < end:
                    return values[i]
        
        def get_region_value_e(number):    
            intervals = [
            (-10, 0.4545), (0.4545, 0.9091), (0.9091, 1.3636),
            (1.3636, 1.8182), (1.8182, 2.2727), (2.2727, 2.7273),
            (2.7273, 3.1818), (3.1818, 3.6364)]
            values=[0,0.5,1,1.5,2,2.5,3,3.5]
            for i, (start, end) in enumerate(intervals):
                if start <= number < end:
                    return values[i]
        def get_region_value_l(number):    
            intervals = [
            (1.0000, 1.3571), (1.3571, 1.7143), (1.7143, 2.0714),
            (2.0714, 2.4286), (2.4286, 2.7857), (2.7857, 3.1429),
            (3.1429, 3.5000)]
            values=[0.5,1,1.5,2,2.5,3,3.5]
            for i, (start, end) in enumerate(intervals):
                if start <= number < end:
                    return values[i]

        if (grade_mode == "fair"):
            num_f=num*5
            return get_region_value_f(num_f)
        elif (grade_mode == "easy"):
            if (num >=0.85):
                return 5
            elif ( 0.85>num>=0.8):
                return 4.5
            elif ( 0.8>num>=0.7):
                return 4
            else:
                num_e = num*5
                return get_region_value_e(num_e)
        elif (grade_mode == "lose_ends"):
            if (num >=0.85):
                return 5
            elif ( 0.85>num>=0.8):
                return 4.5
            elif ( 0.8>num>=0.7):
                return 4
            elif (0.2>=num):
                return 0
            else:
                num_l = num*5
                return get_region_value_l(num_l)
        else:
            print("not a valid mode ")
    
    sa= model.encode(student_answer,convert_to_tensor=True)
    ma= model.encode(model_answer,convert_to_tensor=True)
    grade = Gradanizer(util.cos_sim(sa, ma),mode)
    return grade
    
    

In [23]:
grade = pridector(model = model, student_answer ="love is good for cooking", model_answer = "it's a feeling that make you dumb",mode= "easy")

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

In [24]:
grade

1